In [1]:
import pandas as pd
import main

In [2]:
real_top = pd.read_csv('real_data_test\\top_df.csv', delimiter=';')
real_top = real_top.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_top['X']
del real_top['Y']
real_top.head(n=2)

,WELL_NAME_UWI,DEPTH_TOP
0,100,2548.81
1,101,2556.23


In [3]:
real_bottom = pd.read_csv('real_data_test\\bottom_df.csv', delimiter=';')
real_bottom = real_bottom.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_bottom['X']
del real_bottom['Y']
real_bottom.head(n=2)

,WELL_NAME_UWI,DEPTH_BOTTOM
0,100,2564.81
1,101,2571.71


In [4]:
# данные от организаторов
water_content_data = pd.read_csv(main.data_dir + "\\water_content.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'WC'})
water_content_data = water_content_data.rename(columns={'Дата запуска': 'date_start'})
del water_content_data['date_start']
water_content_data = water_content_data.rename(columns={'well_name': 'WELL_NAME'})
water_content_data.head(n=5)
water_content_data['WELL_NAME'] = water_content_data['WELL_NAME'].astype('str')
water_content_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 2 columns):
WELL_NAME    181 non-null object
WC           181 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.9+ KB


In [5]:
print(len(water_content_data['WELL_NAME'].value_counts()))

180


In [6]:
params_name = ['KINT', 'R05', 'R20', 'R14', 'R10', 'F07', 'F10', 'F14', 'R07', 'F20', 'F05', 'PHIT', 'MRES', 'SG', 'KGL', 'SXWB', 'GZ3', 'NPHI', 'GZ2', 'GZ4', 'GZ1', 'CILD', 'PROX', 'LLD', 'GZ7', 'GR:2', 'GR:1', 'APS', 'KPS', 'GZ5', 'SP:1', 'SP:2', 'W:1', 'W:2', 'NKTM:2', 'NKTM:1', 'CFTC:1', 'CFTC:2', 'MLM:2', 'MLM:1', 'RT:1', 'RT:2', 'GR:3']

In [7]:
las_out_file_name = 'AllGisParams\\fake_gis_las_all_out.csv'
global_data = pd.read_csv(las_out_file_name, delimiter=';')

In [13]:
params_name = ['KINT', 'R05', 'R20', 'R14', 'R10', 'F07', 'F10', 'F14', 'R07', 'F20', 'F05', 'PHIT', 'MRES', 'SG', 'KGL', 'SXWB', 'GZ3', 'NPHI', 'GZ2', 'GZ4', 'GZ1', 'CILD', 'PROX', 'LLD', 'GZ7', 'APS', 'KPS', 'GZ5']
#params_name = ['KINT', 'R05', 'R20', 'R14', 'R10', 'F07', 'F10', 'F14', 'R07', 'F20', 'F05', 'PHIT', 'MRES', 'SG', 'KGL', 'SXWB', 'GZ3', 'NPHI', 'GZ2', 'GZ4', 'GZ1', 'CILD', 'PROX', 'LLD', 'GZ7', 'GR:2', 'GR:1', 'APS', 'KPS', 'GZ5', 'SP:1', 'SP:2', 'W:1', 'W:2', 'NKTM:2', 'NKTM:1', 'CFTC:1', 'CFTC:2', 'MLM:2', 'MLM:1', 'RT:1', 'RT:2', 'GR:3']
result_learning = {}.fromkeys(params_name, 0)

In [15]:
count_val = 100
min_count_val_in_data = 30
for param in params_name:
    print('Convert ', param, 'start')
    param_name_file = (param.replace(':','')).lower()
    # Загружаем данные по параметру
    # las_gis_file_name = 'AllGisParams\\fake_gis_las_' + param_name_file +  '_out.csv'
    # las_data = pd.read_csv(las_gis_file_name, delimiter=';', low_memory=False)
    las_data = global_data[['WELL_NAME', 'WELL_NAME_UWI', 'DEPT', param]].copy()
    las_data['WELL_NAME'] = las_data['WELL_NAME'].astype('str')
    las_data['WELL_NAME_UWI'] = las_data['WELL_NAME_UWI'].astype('str')
    las_data.head(n=5)
    
    # Объединяем данные
    real_data = pd.merge(water_content_data, las_data, on='WELL_NAME')
    real_data = pd.merge(real_data, real_top, on='WELL_NAME_UWI')
    real_data = pd.merge(real_data, real_bottom, on='WELL_NAME_UWI')
    real_data_na = real_data.dropna()
    print(len(real_data['WELL_NAME'].value_counts()))
    
    # Преобразуем в данные для обучения
    dev_path = "..\\tasks\\task 6\\data\\dev\\"
    well_name_list = real_data_na['WELL_NAME_UWI'].value_counts().index.tolist()
    [x_values, y_values, y_names] = main.ConvertDataToLearning(real_data_na, param, dev_path, 10, 100)
    
    y_values_r = [int(10 * y_val) for y_val in y_values]
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import train_test_split, StratifiedKFold
    from sklearn.metrics import accuracy_score
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.cross_validation import cross_val_score

    x_train, x_hold, y_train, y_hold = train_test_split(x_values, y_values_r, test_size=0.3, random_state=17)
    # обучаем дерево
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV, cross_val_score

    forest = RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=17)
    # print(np.mean(cross_val_score(forest, x_hold, y_hold, cv=6))) # 0.949


    forest_params = { 
        'max_depth': [1, 10, 20, 40, 60],
        'n_estimators': [1, 10, 30, 50, 100, 200, 500],
        'max_features': ['auto', 'sqrt', 'log2']
    }

    print('Start', param, ' learning')
    forest_grid = GridSearchCV(forest, forest_params, cv=5, n_jobs=-1, verbose=True)
    forest_grid.fit(x_train, y_train) 
    result_fit = accuracy_score(y_hold, forest_grid.predict(x_hold))
    result_learning[param] = result_fit
    print('Convert', param, 'end')
print(len(x_values))

Convert  KINT start
157
0  of  154
20  of  154
40  of  154
60  of  154
80  of  154
100  of  154
120  of  154
140  of  154


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Start KINT  learning
Fitting 5 folds for each of 105 candidates, totalling 525 fits


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   55.1s
[Parallel(n_jobs=-1)]: Done 525 out of 525 | elapsed:  1.1min finished


Convert KINT end
Convert  R05 start
157
0  of  155
20  of  155
40  of  155
60  of  155
80  of  155
100  of  155
120  of  155
140  of  155
Start R05

...

 learning
Fitting 5 folds for each of 105 candidates, totalling 525 fits
Convert APS end
Convert  KPS start
157
0  of  146
20  of  146
40  of  146
60  of  146
80  of  146
100  of  146
120  of  146
140  of  146
Start KPS  learning
Fitting 5 folds for each of 105 candidates, totalling 525 fits


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   55.8s
[Parallel(n_jobs=-1)]: Done 525 out of 525 | elapsed:  1.1min finished


Convert KPS end
Convert  GZ5 start
157
0  of  156
20  of  156
40  of  156
60  of  156
80  of  156
100  of  156
120  of  156
140  of  156
Start GZ5  learning

D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done 525 out of 525 | elapsed:  1.1min finished



Fitting 5 folds for each of 105 candidates, totalling 525 fits
Convert GZ5 end
153


In [21]:
result_learning

{'APS': 0.069767441860465115,
 'CILD': 0.15217391304347827,
 'F05': 0.2608695652173913,
 'F07': 0.2608695652173913,
 'F10': 0.15217391304347827,
 'F14': 0.28260869565217389,
 'F20': 0.2391304347826087,
 'GZ1': 0.28260869565217389,
 'GZ2': 0.21739130434782608,
 'GZ3': 0.2608695652173913,
 'GZ4': 0.21739130434782608,
 'GZ5': 0.17391304347826086,
 'GZ7': 0.30434782608695654,
 'KGL': 0.17777777777777778,
 'KINT': 0.19565217391304349,
 'KPS': 0.20930232558139536,
 'LLD': 0.19565217391304349,
 'MRES': 0.24444444444444444,
 'NPHI': 0.28888888888888886,
 'PHIT': 0.15555555555555556,
 'PROX': 0.28260869565217389,
 'R05': 0.2391304347826087,
 'R07': 0.28260869565217389,
 'R10': 0.19565217391304349,
 'R14': 0.2608695652173913,
 'R20': 0.19565217391304349,
 'SG': 0.13333333333333333,
 'SXWB': 0.24444444444444444}

In [24]:
#result_learning.items()
result_data = pd.DataFrame(list(result_learning.items()), columns=['Param', 'Result'])

In [28]:
result_data.to_csv('AllGisParams/result_learning.csv',index=False, sep = ';') 

In [9]:
# Создаем DataFrame
# x_names = ['idx' + str(i) for i in range(len(x_values[0]))]
# data_x_values = pd.DataFrame(x_values, columns = x_names)
# data_y_values = pd.DataFrame(y_values, columns=['wc'])
# data_names = pd.DataFrame(y_names, columns=['WELL_NAME'])
# len(data_x_values)
# data_values = pd.merge(data_y_values, data_names, left_index=True, right_index=True)
# data_values = pd.merge(data_values, data_x_values, left_index=True, right_index=True)

In [10]:
data_values.head()

,wc,WELL_NAME,idx0,idx1,idx2,idx3,idx4,idx5,idx6,idx7,...,idx90,idx91,idx92,idx93,idx94,idx95,idx96,idx97,idx98,idx99
0,0.08,554,121.0,130.440,141.60,149.08,150.00,150.000,150.16,152.520,...,171.40,172.760,175.12,176.480,177.00,175.600,170.88,165.160,158.44,151.440
1,0.01,542,106.0,102.125,97.15,90.75,82.80,73.500,65.50,60.300,...,110.75,111.525,112.60,114.075,114.85,116.250,117.00,117.000,117.00,117.000
2,0.33,563,141.5,149.725,153.70,158.10,162.60,164.625,161.80,156.375,...,103.75,117.950,134.40,147.750,154.75,158.125,158.40,154.625,149.40,140.575
3,0.51,319,107.0,112.005,114.51,117.02,120.36,123.025,124.51,125.000,...,113.70,112.515,112.00,112.310,113.98,118.950,121.98,123.990,125.33,126.165
4,0.10,555,156.9,172.300,186.25,195.51,200.56,203.000,203.00,201.990,...,171.10,172.775,174.88,175.970,176.00,176.000,176.78,175.485,172.30,168.700
